# Word2vec implementation for text classification

In [1]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=DeprecationWarning)


In [2]:
import os
import glob
import errno
import json
import numpy as np
import pandas as pd
import nltk

filepath = '../data/bbc/'
labels = []

Accessing the labels names and saving then in "labels_names"

In [3]:
files = os.listdir(filepath)
for name in files:
    if os.path.isdir(filepath+name): 
        labels.append(name)
        
text = []   
texts_aux = [] 
texts_labels = [] 

Extracting the content of each .txt document and label the content of each article.

In [4]:
for label in labels:
    path = filepath+label+'/*.txt'
    files = glob.glob(path)
    for name in files:
        try:
            with open(name, 'r',encoding='ISO-8859-1') as f:
                texts_aux.append(f.read())
                texts_aux.append(label)
        except IOError as exc:
            if exc.errno != errno.EISDIR:
                raise
        texts_labels.append(texts_aux)
        texts_aux=[]

Labeled texts stored in numpy array

In [5]:
texts_labels_np = np.array(texts_labels)

Labeled texts stored in panda dataframe

In [6]:
df = pd.DataFrame(texts_labels, columns=['text','label'])

### Data preparation

Importing necessary dependencies 

In [7]:
# First step: Tokenize each text
from nltk.tokenize import RegexpTokenizer

## Load library for removing stopwords
from nltk.corpus import stopwords
##nltk.download('stopwords') --> First time has to be downloaded

# Import libraries for stemming
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize
stemmer_ps = PorterStemmer()

from nltk.stem.cistem import Cistem
stemmer_cs = Cistem()

# Import lemmatization libraries
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer() 
#nltk.download('wordnet')--> First time has to be downloaded 

# Load stop words 
stop_words = stopwords.words('english')
#print(stop_words[:5])

Create tokenizer and process the raw text

In [8]:
tokenizer = RegexpTokenizer(r'\w+')
texts_clean = []
texts_aux = []
aux = []

for article in texts_labels_np:
        # Text to lower case
        text = article[0].lower()
        # Tokenize and Remove punctuation
        tokens = tokenizer.tokenize(text)
        # Remove stop words
        tokens = [word for word in tokens if word not in stop_words]
        # Stemming
        for token in tokens:
                aux.append(stemmer_cs.stem(token))
        tokens = aux
        
        texts_aux.append(tokens)
        texts_aux.append(article[1])
        texts_clean.append(texts_aux)
        texts_aux = []
        aux=[]

### Data embedding

Transforming labels into numbers [business, entertainment, politics, sport, tech] -- [0,1,2,3,4]

In [9]:
for text in texts_clean:
        if text[1]=='business':
                text[1]=0
        if text[1]=='entertainment':
                text[1]=1
        if text[1]=='politics':
                text[1]=2
        if text[1]=='sport':
                text[1]=3
        if text[1]=='tech':
                text[1]=4

text_clean_np = np.array(texts_clean)
text_clean_pd = pd.DataFrame(texts_labels, columns=['text','label'])

tokenized_texts = []
labels = []
for article in texts_clean:
        tokenized_texts.append(article[0])
        labels.append(article[1])

### Word2vec

Split data into training and test set and prepare it for the doc2vec format.

In [10]:
from sklearn.model_selection import train_test_split
import gensim

# Data file path
dataPath = '../data/'

# Train ratio
train_ratio = 0.85

x_train, x_test, t_train, t_test = train_test_split(tokenized_texts, labels, test_size=1 - train_ratio, stratify=labels)
#x_dev, x_test, t_dev, t_test = train_test_split(x_test, t_test, test_size=0.5, stratify=t_test)

##
##
# Word2Vec
##
##

from  gensim.models.doc2vec import TaggedDocument
from  gensim.models.doc2vec import Doc2Vec


import multiprocessing
cores = multiprocessing.cpu_count()

# We contruct the training and testing dataframe for the word2vec
## Training
words_train = pd.DataFrame(np.array(x_train), columns=['words'])
tags_train = pd.DataFrame(np.array(t_train), columns=['tags'])
documents_train = pd.concat([words_train, tags_train], axis=1)
# Testing
words_test = pd.DataFrame(np.array(x_test), columns=['words'])
tags_test = pd.DataFrame(np.array(t_test), columns=['tags'])
documents_test = pd.concat([words_test, tags_test], axis=1)

In [11]:
# Build the document vectors
def tag_docs(docs):
    tagged = docs.apply(lambda r: gensim.models.doc2vec.TaggedDocument(words=r[0], tags=[r[1]]), axis=1)
    return tagged

# Train the doc2vec model
def train_doc2vec_model(tagged_docs, window, vector_size):
    sents = tagged_docs.values
    doc2vec_model = Doc2Vec(sents, vector_size=vector_size, window=window, epochs=20, dm=0)
    return doc2vec_model

# Construct the final vector feature for the classifier
def vec_for_learning(doc2vec_model, tagged_docs):
    sents = tagged_docs.values
    # Unzipping the values
    targets, regressors = zip(*[(doc.tags[0], doc2vec_model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors


In [12]:
train_tagged = tag_docs(documents_train)
test_tagged = tag_docs(documents_test)


word2vec_model = train_doc2vec_model(train_tagged, 15, 5)


y_train, X_train = vec_for_learning(word2vec_model, train_tagged)
y_test, X_test = vec_for_learning(word2vec_model, test_tagged)

### Logistic regression

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy with LOGISTIC REGRESSION %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score with LOGISTIC REGRESSION: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy with LOGISTIC REGRESSION 0.990990990990991
Testing F1 score with LOGISTIC REGRESSION: 0.9910001805083772


### Random forest

In [14]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=150,n_jobs=1)
rf_model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Testing accuracy with RANDOM FOREST %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score with RANDOM FOREST: {}'.format(f1_score(y_test, y_pred, average='weighted')))

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


NameError: name 'model' is not defined

# SVM

In [ ]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC(C=0.01)
svm_model.fit(X_train, y_train)
print ("Testing accuracy with SVM %s" 
       % accuracy_score(y_test, model.predict(X_test)))

# Assigning unlabeled articles to users by their profile


For this demostration of how the application would function we created a mock up data set of unlabeled articles with two articles for each category and set of users that are defined by the topics they are interested in.


The first user is interested in sports and politics, the second in tech and entertainment,the third in tech and sports and the fourth in business and politics.  Having theseprofiles set, we will test whether the trained model is able to deliver articles that correspond to user topic preferences

In [ ]:
profile1=['sports', 'politics']
profile2=['tech', 'entertainment']
profile3=['tech', 'sports']
profile4=['business','politics']

We first load the text of the articles and label them:

In [ ]:
labels_demo = []
filepath = '../data_demo/'
files = os.listdir(filepath)
for name in files:
    if os.path.isdir(filepath+name): 
        labels_demo.append(name)
        
print(labels_demo)

In [ ]:
text_demo = []   
texts_aux_demo = [] 
texts_labels_demo = [] 
for label in labels_demo:
    path = filepath+label+'/*.txt'
    files = glob.glob(path)
    for name in files:
        try:
            with open(name, 'r',encoding='ISO-8859-1') as f:
                texts_aux_demo.append(f.read())
                texts_aux_demo.append(label)
                print(label)
        except IOError as exc:
            if exc.errno != errno.EISDIR:
                raise
        texts_labels_demo.append(texts_aux_demo)
        texts_aux_demo=[]
        


Articles of **business**:

In [ ]:
for article in texts_labels_demo:
    if article[1]=='business':
        print(article[0])
        print('########')
        print('########')
        print('########')
        print('########')        

Articles for **entertainment**:

In [ ]:
for article in texts_labels_demo:
    
    if article[1]=='entertainment':
        print(article[0])
        print('########')
        print('########')
        print('########')
        print('########')  

Articles for **politics**:

In [ ]:
for article in texts_labels_demo:
    
    if article[1]=='politics':
        print(article[0])
        print('########')
        print('########')
        print('########')
        print('########')  

Articles for **sports**:

In [ ]:
for article in texts_labels_demo:
    
    if article[1]=='sports':
        print(article[0])
        print('########')
        print('########')
        print('########')
        print('########')  

Articles for **tech**:

In [ ]:
for article in texts_labels_demo:
    
    if article[1]=='tech':
        print(article[0])
        print('########')
        print('########')
        print('########')
        print('########')  

The next step is to clean the data and process it for the word2vec embedding algorithm:

In [ ]:
texts_labels_demo_np = np.array(texts_labels_demo)
texts_clean_demo = []
texts_aux_demo = []
aux_demo = []
for article in texts_labels_demo_np:
        # Text to lower case
        text_demo = article[0].lower()
        # Tokenize and Remove punctuation
        tokens = tokenizer.tokenize(text_demo)
        # Remove stop words
        tokens = [word for word in tokens if word not in stop_words]
        # Stemming
        for token in tokens:
                aux_demo.append(stemmer_cs.stem(token))
        tokens = aux_demo
        
        texts_aux_demo.append(tokens)
        texts_aux_demo.append(article[1])
        texts_clean_demo.append(texts_aux_demo)
        texts_aux_demo = []
        aux_demo=[]
print(texts_clean_demo[0])

Transforming labels into numbers [business, entertainment, politics, sport, tech] -- [0,1,2,3,4]

In [ ]:
for text in texts_clean_demo:
        if text[1]=='business':
                text[1]=0
        if text[1]=='entertainment':
                text[1]=1
        if text[1]=='politics':
                text[1]=2
        if text[1]=='sports':
                text[1]=3
        if text[1]=='tech':
                text[1]=4
text_clean_demo_np = np.array(texts_clean_demo)
text_clean_demo_pd = pd.DataFrame(texts_labels_demo, columns=['text','label'])

print(text_clean_demo_np.shape)

tokenized_texts_demo = []
labels_demo = []
for article in texts_clean_demo:
        tokenized_texts_demo.append(article[0])
        labels_demo.append(article[1])
print(len(tokenized_texts_demo))
print(labels_demo)


Now we do the word2vec

In [ ]:

# Construct dataframe
words_test_demo = pd.DataFrame(np.array(tokenized_texts_demo), columns=['words'])
tags_test_demo = pd.DataFrame(np.array(labels_demo), columns=['tags'])
documents_test_demo = pd.concat([words_test_demo, tags_test_demo], axis=1)
print(documents_test_demo)
test_tagged_demo = tag_docs(documents_test_demo)
y_test_demo, X_test_demo = vec_for_learning(word2vec_model, test_tagged_demo)
print(len(y_test_demo))


In [ ]:
# Logistic Regression model
y_pred_demo = logreg.predict(X_test_demo)
print(labels_demo)
print(y_pred_demo)
print('Testing accuracy with LOGISTIC REGRESSION %s' % accuracy_score(y_test_demo, y_pred_demo))
print('Testing F1 score with LOGISTIC REGRESSION: {}'.format(f1_score(y_test_demo, y_pred_demo, average='weighted')))
#Random forest model
y_pred_demo = rf_model.predict(X_test_demo)
print(labels_demo)
print(y_pred_demo)
print('Testing accuracy with RANDOM FOREST %s' % accuracy_score(y_test_demo, y_pred_demo))
print('Testing F1 score with RANDOM FOREST: {}'.format(f1_score(y_test_demo, y_pred_demo, average='weighted')))
# SVM model
y_pred_demo = svm_model.predict(X_test_demo)
print(labels_demo)
print(y_pred_demo)
print('Testing accuracy with SVM %s' % accuracy_score(y_test_demo, y_pred_demo))
print('Testing F1 score with SVM: {}'.format(f1_score(y_test_demo, y_pred_demo, average='weighted')))

The next step would include delivering the classified articles to the respective users. We can see how all the models performce is the same, and all of the them have correctly classiffied 9 out 10 articles from out mock up database. So each user would received the following articles:
   - profile1 --> will received  2 sport articles and 3 politics articles.
   - profile2 --> will received 1 tech article and 2 entertainment articles.
   - profile3 --> will received 1 tech article and 2 entertainment sports.
   - profile4 --> will received  2 business articles and 3 politics articles.


